In [1]:
import os
import sys
import csv
import datetime
import json
import folium

from collections import defaultdict, Counter
from tweepy import API
from tweepy import OAuthHandler
from tweepy import Cursor

In [2]:
def get_twitter_auth():
    consumer_key = 'MqOzyOLVGp5VwMkYvcg839kVS'
    consumer_secret = 'rLCrItfm2csmNm7o9WbCRnkVzJ9MXlKVPxbVQS0cCwgA4cfbgy'
    access_token = '87460496-bq29UUPlMH5GSPEiT7J5TIMkosuwdSWLrTxi22zfT'
    access_secret = 'PxmDK6vrTi30JHjeGHU9yOXviFW80WKroJUvGRGqH97hK'   
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    return auth

def get_twitter_client():
    auth = get_twitter_auth()
    client = API(auth)
    return client

In [3]:
def getHashtags(tweet):
    entities = tweet.get('entities', {})
    hashtags = entities.get('hashtags', [])
    return [tag['text'].lower() for tag in hashtags]

In [4]:
def hashtagAmount(data):
    tags = []
    tagCount = defaultdict(int)
    for hashtag in data['hashtags']:
        try:
            tags.append(hashtag.replace('[', '').replace(']', '').split(','))
        except:
            tags.append([])
    
    for tag in tags:
        amount = len(tag)
        tagCount[amount] += 1
    
    #Calculations
    with_hashtags = sum([count for n_of_tags, count in tagCount.items() if n_of_tags > 0])
    no_hashtags = tagCount[0]
    total = no_hashtags + with_hashtags
    with_hashtags_percent = "%.2f" % (with_hashtags / total * 100)
    no_hashtags_percent = "%.2f" % (no_hashtags / total * 100)
    
    #Print results
    print("{} tweets without hashtags ({}%)".format(no_hashtags, no_hashtags_percent))
    print("{} tweets with at least one hashtag ({}%)".format(with_hashtags, with_hashtags_percent))
    
    for tagCount, tweet_count in tagCount.items():
            if tagCount > 0:
                percent_total = "%.2f" % (tweet_count / total * 100)
                percent_elite = "%.2f" % (tweet_count / with_hashtags * 100)
                print("{} tweets with {} hashtags ({}% total, {}% elite)".format(tweet_count,
                                                                                 tagCount,
                                                                                 percent_total,
                                                                                 percent_elite))

In [5]:
def commonHashtags(fname):
    """
    Get most common hashtags for input jsonl file.
    """
    with open(fname, 'r') as f:
        hashtags = Counter()
        for line in f:
            tweet = json.loads(line)
            hashtags_in_tweet = getHashtags(tweet)
            hashtags.update(hashtags_in_tweet)

        for tag, count in hashtags.most_common(5):
            print('{}: {}'.format(tag, count))

In [6]:
def scrapeTwitter(companies, client):
    for comp in companies:
        fname = '{}_twitter.jsonl'.format(comp)
        with open(fname, 'w') as f:
            for page in Cursor(client.user_timeline, screen_name=comp, count=200).pages(16):
                for status in page:
                    f.write(json.dumps(status._json) + '\n')
    
        makeCSV(fname, comp)
        #os.remove(fname)
        print('{} scraped successfully!\n'.format(comp))

In [7]:
def makeCSV(fname, comp):
    with open(fname, 'r') as f:
        with open('{}_twitter.csv'.format(comp), 'w') as file:
            w = csv.writer(file)
            w.writerow(['company', 'created_at', 'favorite_count', 'retweet_count',
                        'reply_to', 'content', 'hashtags', 'retweet', 'quote'])

            for line in f:
                tweet = json.loads(line)
                company = comp
                
                created_at = datetime.datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
                created_at = created_at.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
                
                favorite_count = tweet['favorite_count']
                retweet_count = tweet['retweet_count']
                reply_to = tweet.get('in_reply_to_screen_name', None)
                content = tweet['text']
                hashtags = None if getHashtags(tweet) == [] else getHashtags(tweet)
                retweet = tweet.get('retweeted', False)
                quote = tweet.get('is_quote_status', False)
                
                w.writerow([company, created_at, favorite_count, retweet_count,
                            reply_to, content, hashtags, retweet, quote])

In [8]:
def makeGeoJson(jsonl_file, name):
    with open(jsonl_file, 'r') as f:
        geo_data = { 
          "type": "FeatureCollection", 
          "features": [] 
        } 
        for line in f:
            tweet = json.loads(line)
            try:
                if tweet['coordinates']:
                    geo_json_feature = { 
                        "type": "Feature", 
                        "geometry": { 
                            "type": "Point", 
                            "coordinates": tweet['coordinates']['coordinates'] 
                            }, 
                        "properties": { 
                            "text": tweet['text'], 
                            "created_at": tweet['created_at'] 
                            } 
                        } 
                    geo_data['features'].append(geo_json_feature) 
            except KeyError: 
                # Skip if json doc is not a tweet (errors, etc.) 
                continue

        with open('{}_twitter_geodata.jsonl'.format(name), 'w') as fout:
            fout.write(json.dumps(geo_data, indent=4))

def make_map(geojson_file, map_file): 
    tweet_map = folium.Map(location=[50, 5], zoom_start=5)
    marker_cluster = folium.MarkerCluster().add_to(tweet_map) 
    geodata = json.load(open(geojson_file)) 
    for tweet in geodata['features']: 
        tweet['geometry']['coordinates'].reverse() 
        marker = folium.Marker(tweet['geometry']['coordinates'], 
                           popup=tweet['properties']['text']) 
        marker.add_to(marker_cluster) 
    return tweet_map 

In [9]:
companies = ['ARENDAutomation', 'BLUMENBECKER', 'Beckhoff_DE', 'Pilz_INT'] #twitter username without @ sign!
client = get_twitter_client()

scrapeTwitter(companies, client)

ARENDAutomation scraped successfully!

BLUMENBECKER scraped successfully!

Beckhoff_DE scraped successfully!

Pilz_INT scraped successfully!

